# Weekly Project 5 
## Implementation of global registration 
### Task 1
Today, your task is to implement a global registration algorithm.

It should be able to roughly align two point clouds.
Implement the global registration, and then try the following:

1. Can you fit `r1.pcd` and `r2.pcd`?
2. Can you fit `car1.ply` and `car2.ply`?
The corresponding files are in the `global_registration` folder.

In [222]:

import open3d as o3d
import numpy as np
import copy

import scipy
from scipy.spatial import cKDTree

In [201]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [202]:
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")

In [203]:
points_numpy = np.asarray(source.points)

In [204]:
points_numpy[0]

array([2.27726722, 1.01953125, 0.88671875])

In [205]:
target

PointCloud with 137833 points.

In [206]:
voxel_size = 0.05

source_downsampled = source.voxel_down_sample(voxel_size)
target_downsampled = target.voxel_down_sample(voxel_size)

In [207]:
max_nn = 100
radius = 20

search_param = o3d.geometry.KDTreeSearchParamHybrid(max_nn, radius)

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_downsampled, search_param)
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_downsampled, search_param)

In [208]:
source_fpfh.num

<bound method PyCapsule.num of Feature class with dimension = 33 and num = 4760
Access its data via data member.>

In [220]:
source_fpfh.data

array([[ 0.        ,  0.        ,  0.        , ...,  1.97964138,
         0.83138983,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         6.15017995,  0.        ],
       [27.76532278, 14.15084564, 14.13441001, ...,  0.        ,
         4.92804121,  0.        ],
       ...,
       [27.68263887, 22.05562451, 32.93382302, ...,  0.        ,
         0.34424572,  0.        ],
       [15.79575165, 18.52599431, 11.27411093, ...,  0.        ,
         2.66272836,  0.        ],
       [12.92155955,  2.76642606,  2.75107944, ...,  0.        ,
        29.25200291,  0.14259685]])

In [223]:
def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
  feat1tree = cKDTree(feat1)
  dists, nn_inds = feat1tree.query(feat0, k=knn, n_jobs=-1)
  if return_distance:
    return nn_inds, dists
  else:
    return nn_inds

In [224]:
def find_correspondences(feats0, feats1, mutual_filter=True):
  nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
  corres01_idx0 = np.arange(len(nns01))
  corres01_idx1 = nns01

  if not mutual_filter:
    return corres01_idx0, corres01_idx1

  nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
  corres10_idx1 = np.arange(len(nns10))
  corres10_idx0 = nns10

  mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
  corres_idx0 = corres01_idx0[mutual_filter]
  corres_idx1 = corres01_idx1[mutual_filter]

  return corres_idx0, corres_idx1

In [ ]:
A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)

In [226]:
matched_indices = find_correspondences(source_fpfh.data, target_fpfh.data)

TypeError: Unexpected keyword argument {'n_jobs': -1}

### Ransac

RANSAC for Pose Estimation
-  Randomly choose 3 pairs of matched points
- Estimate the relative pose
    - Kabsch/ Procrustes
- Apply the transformation and assess its “validity”:
    - number of inliers: matched point pairs whose distance
became “small” after applying the specific transformation
    - Sum of distances between all matched point pairs after
transformation
    - …
- Keep the transformation with most inliers
- Repeat random sampling

In [210]:
def transform_matrix(matrix, transformation):
    matrix_homogeneous = np.hstack([matrix, np.ones((matrix.shape[0], 1))])
    
    transformed_matrix_homogeneous = matrix_homogeneous @ transformation.T
    
    transformed_matrix = transformed_matrix_homogeneous[:, :3]
    
    return transformed_matrix

In [211]:
def calcualte_transformation(source_pc, target_pc):
    source_np = np.asarray(source_pc.points)
    target_np = np.asarray(target_pc.points)
    
    #get random points
    n = 3  # for 2 random indices
    index_source = np.random.choice(source_np.shape[0], n, replace=False)
    index_target = np.random.choice(target_np.shape[0], n, replace=False)
    
    source_points = source_np[index_source]
    target_points = target_np[index_target]

    print("source points")    
    print(source_points)
    print(source_points.shape)
    print("\n")
    print("traget_points")
    print(target_points)
    print(target_points.shape)
    
    #translation
    
    source_centroid = np.mean(source_points[:,-3:], axis=0)
    target_centroid = np.mean(target_points[:,-3:], axis=0)
    
    translation = target_centroid - source_centroid
    # print("translation")
    # print(translation)
    
    cov = source_points.T @ target_points    
    # print(cov)

    
    #rotation
        
    cov_to_sqrt = cov.T @ cov
    
    evalues, evectors = np.linalg.eig(cov_to_sqrt)
    # Ensuring square root matrix exists
    assert (evalues >= 0).all()
    cov_sqrt = evectors * np.sqrt(evalues) @ np.linalg.inv(evectors)
    
    rotation = cov_sqrt @ np.linalg.inv(cov)
    
    # print("\nrotation")
    # print(rotation)
    
    
    transformation = np.eye(4)
    transformation[:3, :3] = rotation
    transformation[:3, 3] = translation
    
    source_points_transformed = transform_matrix(source_points, transformation)
    
    print("\n")
    print("source points tranformed")
    print(source_points_transformed)
    print(source_points_transformed.shape)
    
    return transformation

In [212]:
transformation = calcualte_transformation(source_downsampled, target_downsampled)

# print(transformation)

source points
[[1.37509669 3.60411978 1.6802799 ]
 [1.00682885 3.10071993 1.93354856]
 [1.4582407  2.49905005 2.40228056]]
(3, 3)


traget_points
[[1.20422174 1.50754853 1.07604196]
 [3.09268443 1.3585721  1.89626095]
 [1.94628678 2.60405689 1.09782681]]
(3, 3)


source points tranformed
[[3.45779025 0.96019257 1.75642701]
 [3.48752839 0.76773654 1.11202501]
 [3.23008663 1.37686128 0.5219574 ]]
(3, 3)


In [213]:
source_downsampled_test = np.asarray(source_downsampled.points)[0]

print(source_downsampled_test)

source_downsampled.transform(transformation)

source_downsampled_test = np.asarray(source_downsampled.points)[0]

print(source_downsampled_test)


[2.23442745 1.33203125 2.36328125]
[ 2.15124225  1.96256881 -0.15550935]


In [214]:
draw_registration_result(source_downsampled, target_downsampled, registration_results_point_to_point.transformation)

NameError: name 'registration_results_point_to_point' is not defined

### Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in `car_challange` folder.

You can use the notebooks from Monday as a starting point.